# 1 - Load Keras and evaluate 

In [1]:
import numpy as np
import cv2 
import os
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
key_points_dir = os.path.join("key_points")
keras_weights_dir = os.path.join("keras_weights")

actions = ['sister','hurry','hungry','meal','brother','tree','heavy','cry','family','wise']

In [3]:
# both depends on actions_id -> [0,1,2,3,4,5,6,7,8,9]
def load_dir(dir_name,data_temp=None,labels_temp=None,actions_id=None):
    if actions_id == None or action_id == "all":
        actions_id = [int(s.split('.')[0]) for s in os.listdir(os.path.join(dir_name))]
        actions_id.sort()
    for action_id in actions_id:
        new_array = np.load(os.path.join(dir_name,f"{action_id}.npy"))
        labels_array = np.array([action_id]*len(new_array))

        if(type(data_temp) == np.ndarray):
            data_temp = np.concatenate([data_temp,new_array])
            labels_temp = np.concatenate([labels_temp,labels_array])
        else:
            data_temp = new_array
            labels_temp = labels_array
    
    return data_temp,labels_temp


def load_mul_dir(parent_dir):
    data_temp = None
    labels_temp = None
    for transformation_index in range(len(os.listdir(parent_dir))):
        dir_name = os.path.join(parent_dir,str(transformation_index))
        data_temp,labels_temp = load_dir(dir_name,data_temp,labels_temp)
    return data_temp,labels_temp


train_X,train_Y = load_mul_dir(os.path.join(key_points_dir,"train"))
val_X,val_Y = load_dir(os.path.join(key_points_dir,"val"))
test_X,test_Y = load_dir(os.path.join(key_points_dir,"test"))

perm = np.random.permutation(len(train_X))

train_X = train_X[perm]
train_Y = train_Y[perm]

print(
    train_X.shape,
    train_Y.shape,
    val_X.shape,
    val_Y.shape,
    test_X.shape,
    test_Y.shape
    
)

(7446, 16, 62) (7446,) (190, 16, 62) (190,) (168, 16, 62) (168,)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping


def V1():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(128,return_sequences=True,activation="relu")(layer)
    layer = LSTM(96,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    
    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model



def V2():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(128,return_sequences=True,activation="relu")(layer)
    layer = LSTM(256,return_sequences=True,activation="relu")(layer)
    layer = LSTM(128,return_sequences=True,activation="relu")(layer)
    layer = LSTM(64,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model


def V3():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(256,return_sequences=True,activation="relu")(layer)
    layer = LSTM(96,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    
    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model



def V4():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(512,return_sequences=True,activation="relu")(layer)
    layer = Dropout(0.5)(layer)
    layer = LSTM(128,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    
    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

    
    return model


def V5():
    input_layer = Input(shape=(16,62))
    layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
    layer = LSTM(512,return_sequences=True,activation="relu")(layer)
    layer = LSTM(128,return_sequences=False,activation="relu")(layer)
    layer = Dense(64,activation="relu")(layer)
    layer = Dense(len(actions),activation="softmax")(layer)

    
    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

    
    return model



model = V1()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 62)]          0         
                                                                 
 lstm (LSTM)                 (None, 16, 64)            32512     
                                                                 
 lstm_1 (LSTM)               (None, 16, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 96)                86400     
                                                                 
 dense (Dense)               (None, 64)                6208      
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 224,586
Trainable params: 224,586
Non-trainable

2022-01-06 00:58:18.944483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 00:58:18.948022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 00:58:18.948224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 00:58:18.948541: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
best_model_file_name = os.path.join(keras_weights_dir,"V1.h5")
model.load_weights(best_model_file_name)

In [6]:
print(
    model.evaluate(train_X,train_Y),
    model.evaluate(val_X,val_Y),
    model.evaluate(test_X,test_Y)
)

6/6 [==============================] - 0s 6ms/step - loss: 0.2830 - accuracy: 0.9167
[0.0278821662068367, 0.9924792051315308] [0.14260661602020264, 0.9526315927505493] [0.28297799825668335, 0.9166666865348816]


# save predections

In [8]:
predection = model.predict(test_X)

In [9]:
np.save("predections_test/keras.npy",predection)

In [10]:
np.save("predections_test/labels.npy",test_Y)